# Satellite Image Segmentation Pipeline

This notebook runs the complete pipeline for segmenting Sentinel-2 satellite imagery using SAM.

In [4]:
import os
import sys
sys.path.append("..")
from src.sentinel_preprocessing import preprocess_imagery
from src.sam_satellite_processor import segment_satellite_imagery
from src.polygon_merger import merge_overlapping_segments, concat_polygons

In [8]:
# Define base directory and tile IDs
base_dir = "/home/teulade/images/images_sentinel"
tile_ids = ["32ULU"]
# Construct full paths to tiles
tile_paths = [os.path.join(base_dir, tile_id) for tile_id in tile_ids]
# Define quarters to analyze
quarters_to_analyze = [4]  # Modify this list to choose specific quarters

In [4]:
# Define base directory and tile IDs
base_dir = "/home/teulade/images/Sentinel-2_mosaic_2022"
tile_ids = ["30UVU", "30TXT", "30TYR", "31TCJ", "31TFJ", "31TGL", "31TDM", "31UGP", "31UDP", "31UDR"]
# Construct full paths to tiles
tile_paths = [os.path.join(base_dir, tile_id) for tile_id in tile_ids]

# Define quarters to analyze
quarters_to_analyze = [1, 2, 3, 4]  # Modify this list to choose specific quarters



## Step 1: Preprocess Sentinel-2 Images

In [ ]:
sentinel_bands_paths = [os.path.join(base_dir, tile_id, f"Sentinel-2_mosaic_2022_Q{quarter}_{tile_id}_0_0/") for quarter in quarters_to_analyze for tile_id in tile_ids]

# Preprocess tiles
for sentinel_bands_path in sentinel_bands_paths:
    preprocess_imagery(sentinel_bands_path)

## Step 2: Run SAM Segmentation

In [10]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

sam_checkpoint = "/home/teulade/satellite-sam-segmentation/models/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=10,
    pred_iou_thresh=0.6,
    stability_score_thresh=0.6,
    crop_nms_thresh=0,
    crop_overlap_ratio=1,
    crop_n_layers=1,
    min_mask_region_area=50,
)

/home/teulade/.conda/envs/sat-proj-env/lib/python3.11/site-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(

In [ ]:
# Process tiles
sentinel_bands_paths = [os.path.join(base_dir, tile_id, f"Sentinel-2_mosaic_2022_Q{quarter}_{tile_id}_0_0/") for quarter in quarters_to_analyze for tile_id in tile_ids]

for sentinel_path in sentinel_bands_paths:
    segment_satellite_imagery(sentinel_path, mask_generator, n_samples=20, random_seed=42)

## Step 3: Merge Quarterly Polygons

In [ ]:
# Merge tiles
for tile_path in tile_paths:
    merge_overlapping_segments(tile_path, quarters_to_analyze, 2022)

In [ ]:
concat_polygons(tile_paths, os.path.basename(base_dir))

## Results Visualization

You can add cells here to visualize the results, e.g.:
- Number of polygons per tile
- Sample visualizations of segmented areas
- Processing statistics